In [7]:
import pickle

In [8]:
typos_train = pickle.load( open( "/Users/ozad/Desktop/ecole/masterAic/TC4/tp/projet/typos-data/train20.pkl", "rb" ) )

In [15]:
from numpy import array, ones, zeros, multiply
import numpy as np
import sys
import pprint, math


UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100

class HMM_NS:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print ("HMM creating with: ")
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            print (str(self.N)+" states")
            print (str(self.M)+" observations")
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

    
   
        def forward(self, sentence):
            num_words = len(sentence)
            num_tags = self.N
            trellis = np.zeros((num_words, num_tags))

            for t in range(num_tags):
                tag=t
                word = sentence[0]

                if DO_LOGIFY:
                    trellis[0][t] = self.initial_state_proba[tag] + self.observation_proba[tag][word]
                else:
                    trellis[0][t] = self.initial_state_proba[tag] * self.observation_proba[tag][word]

            for i in range(1, num_words):
                for t in range(num_tags):
                    word = sentence[i]
                    tag=t

                    if DO_LOGIFY:
                        trellis[i][t] = trellis[i - 1][0] + self.transition_proba[0][tag] + self.observation_proba[tag][word]
                        for tt in range(1, num_tags):
                            tag_prev = tt
                            trellis[i][t] = self.log_add(trellis[i][t],
                                                         trellis[i - 1][tt] + self.transition_proba[tag_prev][tag] +
                                                         self.observation_proba[tag][word])
                    else:
                        trellis[i][t] = 0.0
                        for tt in range(num_tags):
                            tag_prev = tt
                            trellis[i][t] += trellis[i - 1][tt] * self.transition_proba[tag_prev][tag] * self.observation_proba[tag][word]

            return trellis

        def backward(self, sentence):
            num_words = len(sentence)
            num_tags = self.N
            trellis = np.zeros((num_words, num_tags))

            for t in range(num_tags):
                if DO_LOGIFY:
                    trellis[num_words - 1][t] = 0.0
                else:
                    trellis[num_words - 1][t] = 1.0

            for i in reversed(range(0, num_words - 1)):
                for t in range(num_tags):
                    word_next = sentence[i + 1]
                    tag=t
                    if DO_LOGIFY:
                        
                        trellis[i][t] = (trellis[i + 1][0] + self.transition_proba[tag][0] +
                                         self.observation_proba[0][word_next])
                        for tt in range(1, num_tags):
                            tag_next = tt
                            trellis[i][t] = self.log_add(trellis[i][t],
                                                         trellis[i + 1][tt] + self.transition_proba[tag][tag_next] +
                                                         self.observation_proba[tag_next][
                                                             word_next])
                    else:
                        trellis[i][t] = 0.0
                        for tt in range(num_tags):
                            tag_next = tt
                            trellis[i][t] += trellis[i + 1][tt] * self.transition_proba[tag][tag_next] * self.observation_proba[tag_next][
                                word_next]

            return trellis

        def get_gamma(self, alpha, beta, i, t):
            if DO_LOGIFY:
                numerator = alpha[i][t] + beta[i][t]
                denominator = alpha[i][0] + beta[i][0]

                for j in range(1, self.N):
                    denominator = self.log_add(denominator, alpha[i][j] + beta[i][j])

                return numerator - denominator
            else:
                numerator = alpha[i][t] * beta[i][t]
                denominator = 0

                for j in range(len(self.tags)):
                    denominator += alpha[i][j] * beta[i][j]

                return numerator / denominator

        def get_epsilon(self, alpha, beta, ti, tj, wt, sentence):
            word = sentence[wt]
            word2 = sentence[wt + 1]
            tag = ti
            tag2 = tj
            if DO_LOGIFY:
                numerator = alpha[wt][ti] + beta[wt + 1][tj] + self.transition_proba[tag][tag2] + self.observation_proba[tag2][word2]
                denominator = (alpha[wt][0] + beta[wt + 1][0] + self.transition_proba[0][0] +
                               self.observation_proba[0][
                                   word2])
                for k in range(self.N):
                    for l in range(self.N):
                        if k == 0 and l == 0:
                            continue
                        k_tag = k
                        l_tag = l
                        denominator = self.log_add(denominator,
                                                   alpha[wt][k] + beta[wt + 1][l] + self.transition_proba[k_tag][l_tag] +
                                                   self.observation_proba[l_tag][
                                                       word2])

                return numerator - denominator
            else:
                numerator = alpha[wt][ti] * beta[wt + 1][tj] * self.transition_proba[tag][tag2] * self.observation_proba[tag2][word2]
                denominator = 0

                for k in range(self.N):
                    for l in range(len(self.tags)):
                        k_tag = k
                        l_tag = l
                        denominator += (alpha[wt][k] * beta[wt + 1][l] * self.transition_proba[k_tag][l_tag] *
                                        self.transition_proba[l_tag][
                                            word2])

                return numerator / denominator

        def baum_welch(self, sentences):
            gamma_cache = []
            epsilon_cache = []
            A= np.zeros( (self.N, self.N) )
            B= np.zeros( (self.M, self.N) ) 

            for sentence in sentences:
                print sentence
                alpha = self.forward(sentence)
                beta = self.backward(sentence)

                gamma = np.zeros((self.N, len(sentence)))
                for wt in range(len(sentence)):
                    for ti in range(self.N):
                        gamma[ti][wt] = self.get_gamma(alpha, beta, wt, ti)
                gamma_cache.append(gamma)

                epsilon = np.zeros((self.N, self.N, len(sentence) - 1))
                for wt in range(len(sentence) - 1):
                    for ti in range(self.N):
                        for tj in range(self.N):
                            epsilon[ti][tj][wt] = self.get_epsilon(alpha, beta, ti, tj, wt, sentence)
                epsilon_cache.append(epsilon)

            
                for i in range(self.N) : 
                    for j in range(self.N) :
                        A[i][j]= np.sum(epsilon[i,j,:])/np.sum(epsilon[i,:,:])
                    
                for j in range(self.N) : 
                    for i in range(self.M) : 
                        for t in range(len(sentence)) : 
                            if sentence[t]==i : 
                                B[i][j] +=gamma[j][t]
                            
                        B[i][j] /= np.sum(gamma[j,:])
                        
                for ti in range(self.N):
                    tag = ti
                    self.initial_state_proba[tag] = 0
                    if DO_LOGIFY:
                        self.initial_state_proba[tag] = math.exp(gamma[ti][0])
                    else:
                        self.initial_state_proba[tag] =gamma[ti][0]

                self.transition_proba = A
                self.observation_proba= B
            

                

        def Viterbi2(self, sent):
            fromStateToLetter = {}
            for i in range(self.N) : 
                fromStateToLetter[i] = np.argmax(self.transition_proba[:][i])
            indexes = self.get_observationIndices(sent)
            delta = self.initial_state_proba * self.observation_proba[indexes[0]]
            N = len(self.initial_state_proba)
            phi = np.zeros((len(indexes), N))
            for count in range(1,len(indexes)):
                delta_inter= np.zeros(N)
                for i in range(N):
                    inter = [delta[j]*self.transition_proba[i][j]*self.observation_proba[indexes[count]][i] for j in range(N)]
                    delta_inter[i] = np.max(inter)
                    phi[count][i] = np.argmax(inter)
                delta = delta_inter
            N_ind = len(indexes)
            etat = np.zeros(N_ind)
            etat[N_ind-1] = np.argmax(delta)
            for i in range(1,N_ind):
                etat[N_ind-1-i] = phi[N_ind-i][etat[N_ind-i]]
            result = []
            for i in range(N_ind):
                result.append(self.omega_X[fromStateToLetter[int(etat[i])]])
            return result

            
        def log_add(self, num_a, num_b):
            if num_b > num_a:
                temp = num_a
                num_a = num_b
                num_b = temp

            if num_a == float('-inf'):
                return num_a

            neg_diff = num_b - num_a
            if neg_diff < -20.0:
                return num_a

            return num_a + math.log(1.0 + math.exp(neg_diff))
        
        def predict(self, typos):
            texte_err = []
            texte_corr = []
            texte_res = []
            for mot in typos:
                mot_err = []
                mot_corr = []
                for letter in mot:
                    letter1 , letter2 = letter
                    mot_err.append(letter1)
                    mot_corr.append(letter2)
                res = hmm.Viterbi2(mot_err)
                texte_err.append(mot_err)
                texte_corr.append(mot_corr)
                texte_res.append(res)
            return (texte_err, texte_corr, texte_res)
            

In [16]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns:
    
    * c_words: word counts
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    c_inits2 = dict()
    c_transitions2 = dict()
    
    for sent in corpus:

            
        if sent[0] in c_inits:
            c_inits[sent[0]] = c_inits[sent[0]] + 1
        else:
            c_inits[sent[0]] = 1
            
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (sent[i-1][1],tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
                    
            if i > 1:
                trans2 = ((sent[i-1][1], sent[i-2][1]), tag)
                if trans2 in c_transitions2:
                    c_transitions2[trans2] = c_transitions2[trans2] + 1
                else:
                    c_transitions2[trans2] = 1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_transitions2, c_inits, c_inits2

def make_vocab(c_words, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    voc.append(UNK)
    for w in c_words:
        if c_words[w] >= threshold:
            voc.append(w)
    return voc


In [ ]:
cwords,ctags,cpairs,ctrans, ctrans2, cinits, cinits2 = make_counts(typos_train)
vocab = make_vocab(cwords, 10)
A_mat = np.ones( (len(ctags.keys()), len(ctags.keys())) )
A_mat = A_mat / np.sum(A_mat,1)
O_mat = np.ones( (len(ctags.keys()), len(vocab)) )
print np.sum(O_mat,1)
O_mat = O_mat / 27
Pi = np.ones( (len(ctags.keys())))
Pi = Pi / np.sum(Pi)

        
        

             
hmm = HMM_NS(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = A_mat,
                 observation_proba = O_mat,
                 initial_state_proba = Pi,
                 smoothing_obs = 0.001)


mots = []
for mot in typos_train : 
    liste = []
    for lettre in mot : 
        liste.append(lettre[1])
    mots.append(hmm.get_observationIndices(liste))
    
DO_LOGIFY=1
             
hmm.baum_welch(mots[1:10])

[ 27.  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.
  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.  27.]
HMM creating with: 
26 states
27 observations
[21  9  4  8 19]
[14 22 15]
[ 1  2  2 14 20 15 21]
[23  8 14 13  4 15  2  4]


In [ ]:
typos_test = pickle.load( open( "/Users/ozad/Desktop/ecole/masterAic/TC4/tp/projet/typos-data/test20.pkl", "rb" ) )

In [13]:
def erreurs(texte_err, texte_corr, texte_res):
    erreur = 0.0
    erreurs_suppr = 0
    erreurs_crees = 0
    length = 0
    for i in range(len(texte_res)):
        length += len(texte_res[i])
        for j in range(len(texte_res[i])):
            if texte_res[i][j] != texte_corr[i][j]:
                erreur += 1
            if texte_err[i][j] != texte_corr[i][j] and texte_res[i][j] == texte_corr[i][j]:
                erreurs_suppr += 1
            if texte_err[i][j] == texte_corr[i][j] and texte_res[i][j] != texte_corr[i][j]:
                erreurs_crees += 1
    erreur = erreur / length
    print("Error rate : ", erreur)
    print("Nombre d'erreurs supprimees ", erreurs_suppr)
    print("Nombre d'erreurs creees : ", erreurs_crees)

In [14]:
resultat = hmm.predict(typos_test)
erreurs(resultat[0], resultat[1], resultat[2])

/Users/ozad/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:270: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


('Error rate : ', 1.0)
("Nombre d'erreurs supprimees ", 0)
("Nombre d'erreurs creees : ", 13452)
